In [22]:
# Import neessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
!pip install scikit-learn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#Connect data
!wget https://github.com/alexeygrigorev/mlbookcamp-code/raw/master/chapter-02-car-price/data.csv


In [23]:
#importing the data
data = pd.read_csv("data.csv")
#data.head()

#treating inconsistenciess, replacing white space with _, and capital letter with small
data.columns=data.columns.str.lower().str.replace(' ','_')
data.head()

for col in data.select_dtypes(include=['object']).columns:
    data[col]=data[col].str.lower().str.replace(' ','_')
    


Data validation setting

In [24]:
# Data preparation
n = len(data)
n_val = n_test = int(n * 0.2)
n_train = n - n_val - n_test
n , n_val+n_test+n_train

data_train = data.iloc[:n_train]
data_val = data.iloc[n_train:n_train + n_val]
data_test = data.iloc[n_train + n_val:]
# Shuffle the dta
idx = np.arange(n)
np.random.shuffle(idx)
#Create shuffled datasets with correct size
data_train = data.iloc[idx[:n_train]]
data_val = data.iloc[idx[n_train:n_train + n_val]]
data_test = data.iloc[idx[n_train + n_val:]]
# reset the index
data_train = data_train.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)
data_test = data_test.reset_index(drop=True)
# apply the log function to y columns
y_train = np.log1p(data_train.msrp.values)
y_val = np.log1p(data_val.msrp.values)
y_test = np.log1p(data_test.msrp.values)

#features preparation 


In [25]:
#Building a regession model
def RG_model(data_train, y_train):
    # Splitting data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_train, y_train, test_size=0.2, random_state=42)

    # Initializing and training the regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predicting on the test set
    y_pred = model.predict(X_test)

    # Calculating RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Printing RMSE
    print(f"Root Mean Squared Error (RMSE): {rmse}")



In [26]:
# Base model
#electing the numeric value only
base = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'popularity']
x_base =data_train[base]
x_base =x_base.fillna(0)

RG_model(x_base, y_train)

Root Mean Squared Error (RMSE): 0.7310854243305847


In [27]:
def preprocess_data(df):
    # Separate features and target variable if 'msrp' is present
    if 'msrp' in df.columns:
        X = df.drop('msrp', axis=1)
        y = df['msrp']
    else:
        X = df
        y = None

    # Identify numerical and categorical columns
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X.select_dtypes(include=['object']).columns

    # Preprocessing for numerical data: impute with mean
    numerical_transformer = SimpleImputer(strategy='mean')

    # Preprocessing for categorical data: one-hot encode
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Preprocess the data
    X_preprocessed = preprocessor.fit_transform(X)

    # If target variable 'msrp' is present, return both X and y
    if y is not None:
        return X_preprocessed, y
    else:
        return X_preprocessed


In [28]:
preprocess_data(data_train)

(<7150x1037 sparse matrix of type '<class 'numpy.float64'>'
 	with 107215 stored elements in Compressed Sparse Row format>,
 0       19120
 1       35370
 2       23155
 3       35445
 4       50950
         ...  
 7145    27499
 7146    32175
 7147    26310
 7148    55285
 7149    28249
 Name: msrp, Length: 7150, dtype: int64)